# 提取并处理数据集MIMIC IV,保存没有los icu和los hosp的数据,且提取数据只有11540，因为去除了包含ICD10的脓毒症患者,用于训练第二个模型

### 从extraction中提取得到的数据，进一步处理从而得到我们丢进模型的信息，而不用每次都需要通过跑一遍代码才能进入模型的训练
#### 通过预处理之后的数据，将数据按照5折交叉验证的方式提取出来，并保存在当前的data_model2文件夹里，其中，保存的数据也按照5折交叉的顺序进行保存，和之前的差异主要是包含了ICD诊断代码
#### 1.生命体征、实验室检测数据
#### 2.去除共存病的静态数据
#### 3.经处理得到的ICD9诊断代码序列
#### 4.知识树，newTree
#### 5.所有的静态数据（包含共存病的静态数据，以防万一，只是保存下来，后面用不用不一定）

In [1]:
# 00-setting
%load_ext autoreload
from __future__ import print_function, division

In [2]:
# 00-配置
%autoreload
# 原始配置
import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import tensorflow as tf
import tensorflow.keras as keras
from keras import backend as K

from utils import *
# 加入官网的配置
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,StratifiedKFold # 分集并5折交叉验证
from sklearn.preprocessing import StandardScaler

In [ ]:
# 01-读入原始数据
ROOT_DIR = 'C:\\Users\\wangyi\\PycharmProjects\\sepsis_mort_icu_master'
DATA_FILEPATH = ROOT_DIR + '/data_mimiciv_icd9'
# RAW_DATA_FILEPATH = '../Extract_output_nogrouping/all_hourly_data.h5'
# GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'stay_id']
# GPU               = '2'
GPU = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
%%time
# 准备数据集
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH + '/vitals_hourly_data.h5')
statics        = pd.read_csv(DATA_FILEPATH + '/static_data.csv')
icd = pd.read_hdf(DATA_FILEPATH + '/C.h5')
newTree = pickle.load(open(ROOT_DIR+'/mimiciv/mimic4/sepsis3_newTree.seqs','rb'))

CPU times: total: 4.22 s
Wall time: 6.18 s


In [5]:
data_full_lvl2

LEVEL2                                Alanine aminotransferase           \
Aggregation Function                                     count mean std   
subject_id hadm_id  stay_id  hours_in                                     
10002155   23822395 33685454 0                             0.0  NaN NaN   
                             1                             0.0  NaN NaN   
                             2                             0.0  NaN NaN   
                             3                             0.0  NaN NaN   
                             4                             0.0  NaN NaN   
...                                                        ...  ...  ..   
19999840   21033226 38978960 123                           0.0  NaN NaN   
                             124                           0.0  NaN NaN   
                             125                           0.0  NaN NaN   
                             126                           0.0  NaN NaN   
                             127                           0.0  NaN NaN   

LEVEL2                                Albumin          Albumin ascites       \
Aggregation Function                    count mean std           count mean   
subject_id hadm_id  stay_id  hours_in                                         
10002155   23822395 33685454 0            0.0  NaN NaN             0.0  NaN   
                             1            0.0  NaN NaN             0.0  NaN   
                             2            0.0  NaN NaN             0.0  NaN   
                             3            0.0  NaN NaN             0.0  NaN   
                             4            0.0  NaN NaN             0.0  NaN   
...                                       ...  ...  ..             ...  ...   
19999840   21033226 38978960 123          0.0  NaN NaN             0.0  NaN   
                             124          0.0  NaN NaN             0.0  NaN   
                             125          0.0  NaN NaN             0.0  NaN   
                             126          0.0  NaN NaN             0.0  NaN   
                             127          0.0  NaN NaN             0.0  NaN   

LEVEL2                                    Albumin pleural  ...  \
Aggregation Function                  std           count  ...   
subject_id hadm_id  stay_id  hours_in                      ...   
10002155   23822395 33685454 0        NaN             0.0  ...   
                             1        NaN             0.0  ...   
                             2        NaN             0.0  ...   
                             3        NaN             0.0  ...   
                             4        NaN             0.0  ...   
...                                    ..             ...  ...   
19999840   21033226 38978960 123      NaN             0.0  ...   
                             124      NaN             0.0  ...   
                             125      NaN             0.0  ...   
                             126      NaN             0.0  ...   
                             127      NaN             0.0  ...   

LEVEL2                                White blood cell count  \
Aggregation Function                                     std   
subject_id hadm_id  stay_id  hours_in                          
10002155   23822395 33685454 0                           NaN   
                             1                           NaN   
                             2                           NaN   
                             3                           NaN   
                             4                           NaN   
...                                                      ...   
19999840   21033226 38978960 123                         0.0   
                             124                         NaN   
                             125                         NaN   
                             126                         NaN   
                             127                         NaN   

LEVEL2              

In [6]:
data_full_lvl2.head()

LEVEL2                                Alanine aminotransferase           \
Aggregation Function                                     count mean std   
subject_id hadm_id  stay_id  hours_in                                     
10002155   23822395 33685454 0                             0.0  NaN NaN   
                             1                             0.0  NaN NaN   
                             2                             0.0  NaN NaN   
                             3                             0.0  NaN NaN   
                             4                             0.0  NaN NaN   

LEVEL2                                Albumin          Albumin ascites       \
Aggregation Function                    count mean std           count mean   
subject_id hadm_id  stay_id  hours_in                                         
10002155   23822395 33685454 0            0.0  NaN NaN             0.0  NaN   
                             1            0.0  NaN NaN             0.0  NaN   
                             2            0.0  NaN NaN             0.0  NaN   
                             3            0.0  NaN NaN             0.0  NaN   
                             4            0.0  NaN NaN             0.0  NaN   

LEVEL2                                    Albumin pleural  ...  \
Aggregation Function                  std           count  ...   
subject_id hadm_id  stay_id  hours_in                      ...   
10002155   23822395 33685454 0        NaN             0.0  ...   
                             1        NaN             0.0  ...   
                             2        NaN             0.0  ...   
                             3        NaN             0.0  ...   
                             4        NaN             0.0  ...   

LEVEL2                                White blood cell count  \
Aggregation Function                                     std   
subject_id hadm_id  stay_id  hours_in                          
10002155   23822395 33685454 0                           NaN   
                             1                           NaN   
                             2                           NaN   
                             3                           NaN   
                             4                           NaN   

LEVEL2                                White blood cell count urine           \
Aggregation Function                                         count mean std   
subject_id hadm_id  stay_id  hours_in                                         
10002155   23822395 33685454 0                                 0.0  NaN NaN   
                             1                                 0.0  NaN NaN   
                             2                                 0.0  NaN NaN   
                             3                                 0.0  NaN NaN   
                             4                                 0.0  NaN NaN   

LEVEL2                                   pH          pH urine           
Aggregation Function                  count mean std    count mean std  
subject_id hadm_id  stay_id  hours_in                                   
10002155   23822395 33685454 0          0.0  NaN NaN      0.0  NaN NaN  
                             1          0.0  NaN NaN      0.0  NaN NaN  
                             2          0.0  NaN NaN      0.0  NaN NaN  
                             3          0.0  NaN NaN      0.0  NaN NaN  
                             4          0.0  NaN NaN      0.0  NaN NaN  

[5 rows x 222 columns]

In [7]:
statics.head()

,subject_id,hadm_id,stay_id,intime,outtime,age,gender,is_male,race,race_white,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,81.592179,F,0,WHITE,1,...,0,0,0,1,0,0,0,0,0,0
1,10002428,28662225,38875437,2156-04-19 18:11:19,2156-04-26 18:58:41,81.280232,F,0,WHITE,1,...,1,0,0,1,0,0,0,0,0,0
2,10003400,20214994,32128372,2137-02-25 23:37:19,2137-03-10 21:29:36,75.149738,F,0,BLACK/AFRICAN AMERICAN,0,...,1,1,1,0,0,0,0,0,0,0
3,10004235,24181354,34100191,2196-02-24 17:07:00,2196-02-29 15:58:02,47.149517,M,1,BLACK/CAPE VERDEAN,0,...,1,0,0,1,0,0,0,0,0,0
4,10004401,29988601,32773003,2144-01-26 22:28:04,2144-02-06 13:44:15,85.059155,M,1,WHITE,1,...,0,0,0,1,0,0,1,0,0,0


In [8]:
icd.head()

,,,icd9_codes
subject_id,hadm_id,stay_id,
12207593,22795209,30000646,"[19889 , 51881 , 41519 , 4233 , 78551 , ..."
16513856,24463832,30001446,"[03849 , 78552 , 5849 , 07044 , 5990 , ..."
15332791,20683754,30003598,"[99662 , 0388 , 486 , 5070 , 1179 , ..."
10369174,24697158,30004144,"[43310 , 5780 , 53020 , 11284 , 43330 , ..."
17220323,25700666,30004242,"[85220 , 3485 , 4019 , 2720 , 2440 , ..."


In [9]:
# 首先处理得到ICD9诊断代码的序列
# 从单级CCS分类文件中获取ICD到CCS的映射
def icdMapper(ccsFolder):
    #singleFile = open(os.path.join(ccsFolder, '$dxref 2015.csv'), 'r')   # 单级CCS文本
    singleFile = open(ccsFolder,'r')
    cnt = 0
    icd2Cid = {}        # len(icd2Cid)=15072
    cidSet = set()      # len(cidSet)=283
    singleFile.readline()
    singleFile.readline()
    singleFile.readline()
    for l in singleFile:    # ICD9,CCS,CCS描述,ICD9描述（,OptionalCCS,OptionalCCS描述）---procedure(diagnoses)
        tokens = l.strip().split(',')
        icd = tokens[0][1:-1].strip()   # ICD9
        curCid = int(tokens[1][1:-1].strip())   # ccs
        icd2Cid[icd] = curCid
    singleFile.close()
    return icd2Cid  # 返回的是ICD9:CCS单级编码映射的字典

# 读取ccs文件
ccsFolder = '../ccs'
icd2Cid = icdMapper(ccsFolder+'/$dxref 2015.csv')   # 单级ccs映射

In [10]:
def icdMappers(icds):
    ccs=[]
    for icd in icds:
        ccs.append(icd2Cid[icd.strip()])
    return ccs
to_ccs = lambda x: icdMappers(x)


In [11]:
icd['ccs_codes'] = (icd['icd9_codes']).apply(to_ccs)

In [12]:
icd.head()

,,,icd9_codes,ccs_codes
subject_id,hadm_id,stay_id,,
12207593,22795209,30000646,"[19889 , 51881 , 41519 , 4233 , 78551 , ...","[42, 131, 103, 97, 249, 42, 58, 97, 19, 133, 4..."
16513856,24463832,30001446,"[03849 , 78552 , 5849 , 07044 , 5990 , ...","[2, 249, 157, 6, 159, 55, 151, 6, 259, 62, 151..."
15332791,20683754,30003598,"[99662 , 0388 , 486 , 5070 , 1179 , ...","[237, 2, 122, 129, 4, 155, 145, 159, 238, 135,..."
10369174,24697158,30004144,"[43310 , 5780 , 53020 , 11284 , 43330 , ...","[110, 153, 138, 4, 110, 163, 163, 162, 2617, 2..."
17220323,25700666,30004242,"[85220 , 3485 , 4019 , 2720 , 2440 , ...","[233, 95, 98, 53, 48, 55, 2620, 2621]"


In [13]:
def toIndex(lawSeqs):
    types = {}
    newSeqs = []
    for patients in lawSeqs:
        newPa = []
        for code in patients:
            if type(code) == str:
                code = code.strip()
            if code in types:
                newPa.append(types[code])
            else:
                types[code] = len(types)
                newPa.append(types[code])
        newSeqs.append(newPa)
    return types,newSeqs

In [14]:
'''
    ccs相关的：seqs(原始的),types（原始到索引的映射字典）,newSeqs（以单级ccs的索引为主的ccs诊断序列）
    icd9相关的：seqs_icd（原始的）,types_icd（原始到索引的映射字典）,newSeqs_icd（以ICD9诊断代码为主的序列）
'''
seqs_icd = icd['icd9_codes'].values
seqs = icd['ccs_codes'].values
types={}
types_icd={}
newSeqs = []
newSeqs_icd =[]
types,newSeqs = toIndex(seqs)
types_icd,newSeqs_icd = toIndex(seqs_icd)
icd['ccs']=newSeqs
icd['icd9'] = newSeqs_icd

In [15]:
icd.head()

,,,icd9_codes,ccs_codes,ccs,icd9
subject_id,hadm_id,stay_id,,,,
12207593,22795209,30000646,"[19889 , 51881 , 41519 , 4233 , 78551 , ...","[42, 131, 103, 97, 249, 42, 58, 97, 19, 133, 4...","[0, 1, 2, 3, 4, 0, 5, 3, 6, 7, 0, 8, 2, 9, 10,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
16513856,24463832,30001446,"[03849 , 78552 , 5849 , 07044 , 5990 , ...","[2, 249, 157, 6, 159, 55, 151, 6, 259, 62, 151...","[16, 4, 17, 18, 19, 20, 21, 18, 22, 23, 21, 16...","[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3..."
15332791,20683754,30003598,"[99662 , 0388 , 486 , 5070 , 1179 , ...","[237, 2, 122, 129, 4, 155, 145, 159, 238, 135,...","[25, 16, 26, 27, 28, 29, 30, 19, 31, 32, 33, 3...","[34, 35, 36, 37, 38, 39, 40, 24, 41, 42, 43, 4..."
10369174,24697158,30004144,"[43310 , 5780 , 53020 , 11284 , 43330 , ...","[110, 153, 138, 4, 110, 163, 163, 162, 2617, 2...","[42, 43, 44, 28, 42, 45, 45, 46, 12, 12, 44, 4...","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6..."
17220323,25700666,30004242,"[85220 , 3485 , 4019 , 2720 , 2440 , ...","[233, 95, 98, 53, 48, 55, 2620, 2621]","[53, 54, 36, 10, 37, 20, 55, 56]","[80, 81, 46, 14, 82, 83, 84, 85]"


In [26]:
# 03-数据处理1-更改版本，硬算-4.1(试这个)
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))

    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()

    df_out.loc[:, idx[:, 'mean']] = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(method='bfill').groupby(ID_COLS).fillna(icustay_means)#.fillna(0)

    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)

#     计算time_since_measured,优化
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.groupby(ID_COLS).cumsum()    # 就是这里有错误，应该是按照ID_COLS来计算hours_of_absence
#     time_since_measured = hours_of_absence - hours_of_absence[is_absent == 0].fillna(method='ffill')
    time_since_measured = hours_of_absence-hours_of_absence[is_absent==0].groupby(ID_COLS).fillna(method='ffill').groupby(ID_COLS).fillna(1)
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)

    df_out.sort_index(axis=1, inplace=True)
    return df_out

In [20]:
# 03-数据处理2
statics = statics.set_index(['subject_id', 'hadm_id', 'stay_id'])
Ys = statics[['mort_icu', 'mort_hosp','thirtyday_expire_flag','icu_los','hosp_los','readmission_30']]
Ys.astype(float)
stas = statics.drop(columns=['mort_icu', 'mort_hosp','thirtyday_expire_flag','icu_los','hosp_los','readmission_30','gender','race','admittime','dischtime','discharge_location','deathtime','icu_los','hosp_los','intime','outtime'])

stas = pd.get_dummies(stas, columns=['insurance','admission_type','first_careunit'])
stas = stas.fillna(0)

statics['icd9'] = icd['icd9']
statics['newTree'] = newTree
# 不含共存病的静态信息
stasNoGCB = stas[['age','is_male','race_white','race_black','race_yellow','race_hispanic','race_other','weight','urineoutput','gcs_eyes','gcs_verbal','gcs_motor','hosp_adm_time','insurance_Medicaid','insurance_Medicare','insurance_Other' ,'admission_type_AMBULATORY OBSERVATION','admission_type_DIRECT EMER.','admission_type_DIRECT OBSERVATION','admission_type_ELECTIVE','admission_type_EU OBSERVATION','admission_type_EW EMER.','admission_type_OBSERVATION ADMIT','admission_type_SURGICAL SAME DAY ADMISSION','admission_type_URGENT','first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)','first_careunit_Coronary Care Unit (CCU)','first_careunit_Medical Intensive Care Unit (MICU)','first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)','first_careunit_Surgical Intensive Care Unit (SICU)','first_careunit_Trauma SICU (TSICU)']]

lvl2 = data_full_lvl2[
    (data_full_lvl2.index.get_level_values('stay_id').isin(set(Ys.index.get_level_values('stay_id')))) &
    (data_full_lvl2.index.get_level_values('hours_in') < WINDOW_SIZE)
]
# 使用索引实现分层分割
train_frac,test_frac = 0.8,0.2

lvl2_subj_idx, static_subj_idx, Ys_subj_idx,Y_mort_icu_0_subj_idx,Y_mort_icu_1_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, stas, Ys,Ys[Ys['mort_icu']==0],Ys[Ys['mort_icu']==1])]
lvl2_subjects = set(lvl2_subj_idx)
neg_subjects,pos_subjects = set(Y_mort_icu_0_subj_idx),set(Y_mort_icu_1_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"
assert lvl2_subjects == set(static_subj_idx), "Subject ID pools differ!"

np.random.seed(SEED)
neg_subjects,pos_subjects,neg_N,pos_N = np.random.permutation(list(neg_subjects)),np.random.permutation(list(pos_subjects)),len(neg_subjects),len(pos_subjects)
# subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)
neg_N_train, neg_N_test,pos_N_train, pos_N_test = int(train_frac * neg_N), int(test_frac * neg_N),int(train_frac * pos_N), int(test_frac * pos_N)
neg_train_subj = neg_subjects[:neg_N_train]
neg_test_subj  = neg_subjects[neg_N_train:]
pos_train_subj = pos_subjects[:pos_N_train]
pos_test_subj  = pos_subjects[pos_N_train:]
train_subj = np.append(neg_train_subj,pos_train_subj,axis=0)
test_subj = np.append(neg_test_subj,pos_test_subj,axis=0)

[(lvl2_train, lvl2_test), (stasNoGCB_train, stasNoGCB_test), (icd_train, icd_test),(tree_train, tree_test), (Ys_train, Ys_test),(stas_train, stas_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, test_subj)] for df in (lvl2, stasNoGCB, statics['icd9'], statics['newTree'], Ys['mort_icu'], stas)
]

In [8]:
statics = statics.set_index(['subject_id', 'hadm_id', 'stay_id'])
Ys = statics[['mort_icu', 'mort_hosp','thirtyday_expire_flag','icu_los','hosp_los','readmission_30']]
Ys.astype(float)
lvl2 = data_full_lvl2[
    (data_full_lvl2.index.get_level_values('stay_id').isin(set(Ys.index.get_level_values('stay_id')))) &
    (data_full_lvl2.index.get_level_values('hours_in') < WINDOW_SIZE)
]

In [9]:
lvl2

LEVEL2                                Alanine aminotransferase           \
Aggregation Function                                     count mean std   
subject_id hadm_id  stay_id  hours_in                                     
10002155   23822395 33685454 0                             0.0  NaN NaN   
                             1                             0.0  NaN NaN   
                             2                             0.0  NaN NaN   
                             3                             0.0  NaN NaN   
                             4                             0.0  NaN NaN   
...                                                        ...  ...  ..   
19999840   21033226 38978960 19                            0.0  NaN NaN   
                             20                            0.0  NaN NaN   
                             21                            0.0  NaN NaN   
                             22                            0.0  NaN NaN   
                             23                            0.0  NaN NaN   

LEVEL2                                Albumin          Albumin ascites       \
Aggregation Function                    count mean std           count mean   
subject_id hadm_id  stay_id  hours_in                                         
10002155   23822395 33685454 0            0.0  NaN NaN             0.0  NaN   
                             1            0.0  NaN NaN             0.0  NaN   
                             2            0.0  NaN NaN             0.0  NaN   
                             3            0.0  NaN NaN             0.0  NaN   
                             4            0.0  NaN NaN             0.0  NaN   
...                                       ...  ...  ..             ...  ...   
19999840   21033226 38978960 19           0.0  NaN NaN             0.0  NaN   
                             20           0.0  NaN NaN             0.0  NaN   
                             21           0.0  NaN NaN             0.0  NaN   
                             22           0.0  NaN NaN             0.0  NaN   
                             23           0.0  NaN NaN             0.0  NaN   

LEVEL2                                    Albumin pleural  ...  \
Aggregation Function                  std           count  ...   
subject_id hadm_id  stay_id  hours_in                      ...   
10002155   23822395 33685454 0        NaN             0.0  ...   
                             1        NaN             0.0  ...   
                             2        NaN             0.0  ...   
                             3        NaN             0.0  ...   
                             4        NaN             0.0  ...   
...                                    ..             ...  ...   
19999840   21033226 38978960 19       NaN             0.0  ...   
                             20       NaN             0.0  ...   
                             21       NaN             0.0  ...   
                             22       NaN             0.0  ...   
                             23       NaN             0.0  ...   

LEVEL2                                White blood cell count  \
Aggregation Function                                     std   
subject_id hadm_id  stay_id  hours_in                          
10002155   23822395 33685454 0                           NaN   
                             1                           NaN   
                             2                           NaN   
                             3                           NaN   
                             4                           NaN   
...                                                      ...   
19999840   21033226 38978960 19                          NaN   
                             20                          NaN   
                             21                          NaN   
                             22                          NaN   
                             23                          NaN   

LEVEL2              

In [23]:
# K折交叉验证
cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
cv_idx = list(cv.split(X=np.zeros(Ys_train.shape[0]), y=Ys_train.values))
# 计算fold mean和fold std
fold_mean = []
fold_std = []
idx = pd.IndexSlice
for train, validation in cv_idx:
    fold_train_subject_id=[]
#     fold_val_subject_id=[]
    for i in train:
        fold_train_subject_id.append(stas_train.index[i][0])
#     for i in validation:
#         fold_val_subject_id.append(stas_train.index[i][0])

    fold = lvl2_train[lvl2_train.index.get_level_values('subject_id').isin(fold_train_subject_id)]
#     [fold] = [[df[df.index.get_level_values('subject_id').isin(s)] for s in fold_train_subject_id] for df in lvl2_train]

    mean,std = fold.loc[:,idx[:,'mean']].mean(axis=0),fold.loc[:,idx[:,'mean']].std(axis=0)
    #Store training fold mean and std to standardize validation folds later
    #Extra dimension is added to make broadcasted division and subtraction possible
    fold_mean.append(mean)
    fold_std.append(std)

In [27]:
# 模型设置
X_train = simple_imputer(lvl2_train)  # 应该在归一化处理之后使用
X_test = simple_imputer(lvl2_test)
structNoGCB_train = stasNoGCB_train.values
structNoGCB_test = stasNoGCB_test.values
ICD_train = icd_train.values
ICD_test = icd_test.values
newTree_train = tree_train.values
newTree_test = tree_test.values
struct_train = stas_train.values
struct_test = stas_test.values
y_train = Ys_train.values
y_test = Ys_test.values
X_train_3d = to_3D_tensor(X_train)
X_test_3d = to_3D_tensor(X_test)

mask_train = X_train_3d[:,np.arange(0,X_train_3d.shape[1],3),:]
measurement_train = X_train_3d[:,np.arange(1,X_train_3d.shape[1],3),:]
time_train = X_train_3d[:,np.arange(2,X_train_3d.shape[1],3),:]
mask_test = X_test_3d[:,np.arange(0,X_test_3d.shape[1],3),:]
measurement_test = X_test_3d[:,np.arange(1,X_test_3d.shape[1],3),:]
time_test = X_test_3d[:,np.arange(2,X_test_3d.shape[1],3),:]

C:\Users\spring\AppData\Local\Temp\ipykernel_20660\1004335290.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.loc[:, idx[:, 'mean']] = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).fillna(
C:\Users\spring\AppData\Local\Temp\ipykernel_20660\1004335290.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
C:\Users\spring\AppData\Local\Temp\ipykernel_20660\1004335290.py:15: SettingWithCopyWarning: 
A value is trying to 

In [29]:
# 保存5折交叉数据集

folder_data = ROOT_DIR+'/mimiciv/data_fold5'    # 直接保存到第二个模型的文件夹下

fold_trainSet = []   # 预测变量
fold_validSet = []
fold_testSet = []
fold_Y = []    # Y=(train,valid)
fold_Xmean = []

i = 0
for train, validation in cv_idx:
    print(f'Fold {i}. Positive examples in validation: {np.sum(y_train[validation])}...')

    # 归一化
    scaled_X_train = (measurement_train[train] - fold_mean[i][None,:,None])/fold_std[i][None,:,None] #Standardize
    scaled_X_train = np.nan_to_num(scaled_X_train) #Implicit mean imputation for remaining NaNs
    scaled_X_validation = (measurement_train[validation] - fold_mean[i][None,:,None])/fold_std[i][None,:,None] #Standardize using train set mean. No Leaking
    scaled_X_validation = np.nan_to_num(scaled_X_validation) #Implicit mean imputation for remaining NaNs
    scaled_X_test = (measurement_test-fold_mean[i][None,:,None])/fold_std[i][None,:,None]
    scaled_X_test = np.nan_to_num(scaled_X_test) #Implicit mean imputation for remaining NaNs
    #last_obsv
    measurement_last_obsv = scaled_X_train.transpose([0,2,1])
    measurement_last_obsv_validation = scaled_X_validation.transpose([0,2,1])
    measurement_last_obsv_test = scaled_X_test.transpose([0,2,1])
    # mask & time
    mask_train_ = mask_train[train]
    mask_validation = mask_train[validation]
    time_train_ = time_train[train]
    time_validation = time_train[validation]
    # 计算X_mean
    X_mean = np.nanmean(np.where(mask_train_==1,1,np.NaN) * scaled_X_train,axis=0,keepdims=True).transpose([0,2,1])

    # struct
    structNoGCB_train_ = structNoGCB_train[train]
    structNoGCB_validation = structNoGCB_train[validation]

    struct_train_ = struct_train[train]
    struct_validation = struct_train[validation]

    # ICD9序列
    ICD_train_ = ICD_train[train]
    ICD_validation = ICD_train[validation]

    # 知识图谱之知识树
    newTree_train_ = newTree_train[train]
    newTree_validation = newTree_train[validation]

    trainSet = (scaled_X_train.transpose([0,2,1]),measurement_last_obsv,mask_train_.transpose([0,2,1]),time_train_.transpose([0,2,1]))
    validSet = (scaled_X_validation.transpose([0,2,1]),measurement_last_obsv_validation,mask_validation.transpose([0,2,1]),time_validation.transpose([0,2,1]))
    testSet = (scaled_X_test.transpose([0,2,1]),measurement_last_obsv_test,mask_test.transpose([0,2,1]),time_test.transpose([0,2,1]))

    fold_trainSet.append((trainSet,structNoGCB_train_,ICD_train_,newTree_train_,struct_train_))
    fold_validSet.append((validSet,structNoGCB_validation,ICD_validation,newTree_validation,struct_validation))
    fold_testSet.append((testSet,structNoGCB_test,ICD_test,newTree_test,struct_test))   # 用于后面保存每折叠的测试集
    fold_Xmean.append(X_mean)
    fold_Y.append((y_train[train],y_train[validation]))

# 保存训练数据
# pickle.dump(fold_trainSet,open(os.path.join(folder_data,'fold_Set.train'), 'wb'),-1)
# pickle.dump(fold_testSet,open(os.path.join(folder_data,'fold_Set.test'), 'wb'),-1)
# pickle.dump(fold_validSet,open(os.path.join(folder_data,'fold_Set.valid'), 'wb'),-1)
# pickle.dump(fold_Xmean,open(os.path.join(folder_data,'fold_Set.Xmean'), 'wb'),-1)
# pickle.dump(fold_Y,open(os.path.join(folder_data,'fold_Set.labels'), 'wb'),-1)
# pickle.dump(y_test,open(os.path.join(folder_data,'test.labels'), 'wb'),-1)


Fold 0. Positive examples in validation: 204...


C:\Users\spring\AppData\Local\Temp\ipykernel_20660\2825106442.py:18: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  scaled_X_train = (measurement_train[train] - fold_mean[i][None,:,None])/fold_std[i][None,:,None] #Standardize
C:\Users\spring\AppData\Local\Temp\ipykernel_20660\2825106442.py:20: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  scaled_X_validation = (measurement_train[validation] - fold_mean[i][None,:,None])/fold_std[i][None,:,None] #Standardize using train set mean. No Leaking
C:\Users\spring\AppData\Local\Temp\ipykernel_20660\2825106442.py:22: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array befor

Fold 0. Positive examples in validation: 203...
Fold 0. Positive examples in validation: 204...
Fold 0. Positive examples in validation: 204...
Fold 0. Positive examples in validation: 204...
